In [1]:
#####可変基底非負値テンソル分解#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
import gc
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    if pattern==1:
        z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id

In [3]:
####データの生成####
##データと日付idの設定
#データの設定
k = 10
k_vec = np.repeat(1.0, k)
mode = 3
m = 3
hh = 2500
location = 1500
week = 7
month = 31
year = 12
max_week = 5
month_days = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
year_days = np.sum(month_days)

#日付IDを設定
day_id = np.tile(np.arange(week), int((year_days*m)/week) + 1)[:m*year_days]
week_id = np.tile(np.array(list(itertools.chain(*[np.repeat(np.arange(max_week), week)[:month_days[j]] for j in range(year)]))), m)
month_id = np.tile(np.repeat(np.arange(year), month_days), m)
date_id = [day_id, week_id, month_id]
date_n = np.array([week, max_week, year])

In [7]:
##パラメータの生成
#事前分布の設定
gamma = np.array([3.0, 1.0, 0.25, 0.5])
alpha1 = np.array([0.3]); alpha2 = np.array([0.15])
beta1 = np.array([0.75]); beta2 = np.array([0.3])

#モデルパラメータの生成
omega = np.random.dirichlet(gamma, hh)
theta_u = np.random.gamma(alpha1, 1/beta1, hh*k).reshape(hh, k)
theta_v = np.random.gamma(alpha1, 1/beta1, location*k).reshape(location, k)
theta_d = [j for j in range(mode+1)]
for j in range(mode):
    theta_d[j] = np.random.gamma(alpha2, 1/beta2, date_n[j]*k).reshape(date_n[j], k)
theta_d[mode] = np.repeat(1.0, k)
omegat = omega.copy()
thetat_u = theta_u.copy(); thetat_v = theta_v.copy(); thetat_d = theta_d.copy()

In [8]:
##応答変数を生成
#データの設定
n = location*m*year_days
freq_par = np.random.gamma(m*10, 1/0.15, hh)
location_pattern = np.tile(np.arange(location), m*year_days)
date_pattern = [np.repeat(date_id[j], location) for j in range(mode)]

#結合パラメータを定義
joint_theta = [j for j in range(m+1)]
for j in range(mode+1):
    if j < mode:
        joint_theta[j] = theta_v[location_pattern, ] * theta_d[j][date_pattern[j], ]
    else:
        joint_theta[j] = theta_v[location_pattern, ]

#データの格納用配列
z_list = [i for i in range(hh)]
y_list = [i for i in range(hh)]
d_list = [i for i in range(hh)]
location_list = [i for i in range(hh)]
day_list = [i for i in range(hh)]
week_list = [i for i in range(hh)]
month_list = [i for i in range(hh)]
pt = np.repeat(0, hh)

for i in range(hh):
    if i%100==0:
        print(i)
        
    #全パターンの頻度を生成
    z = np.random.multinomial(1, omega[i, ], n)
    Lambda = np.zeros((n, mode+1))
    for j in range(mode+1):
        Lambda[:, j] = np.dot(theta_u[i, ] * joint_theta[j], k_vec)
    y_freq = np.random.poisson(np.sum(z * Lambda, axis=1))
    
    #データをサンプリング
    index_nonzero = np.where(y_freq > 0)[0]
    index_freq = np.where(np.random.binomial(1, freq_par[i]/index_nonzero.shape[0], index_nonzero.shape[0]))[0]
    index_target = index_nonzero[index_freq]
    
    #リストに格納
    z_list[i] = z[index_target, ]
    y_list[i] = y_freq[index_target]
    d_list[i] = np.repeat(i, index_target.shape[0])
    location_list[i] = location_pattern[index_target]
    day_list[i] = date_pattern[0][index_target]
    week_list[i] = date_pattern[1][index_target]
    month_list[i] = date_pattern[2][index_target]
    pt[i] = index_target.shape[0]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


In [ ]:
#リストを変換
z = np.array(list(itertools.chain(*[z_list[i] for i in range(hh)])))
y = np.array(list(itertools.chain(*[y_list[i] for i in range(hh)])))
d_id = np.array(list(itertools.chain(*[d_list[i] for i in range(hh)])))
location_id = np.array(list(itertools.chain(*[location_list[i] for i in range(hh)])))
day_id = np.array(list(itertools.chain(*[day_list[i] for i in range(hh)])))
week_id = np.array(list(itertools.chain(*[week_list[i] for i in range(hh)])))
month_id = np.array(list(itertools.chain(*[month_list[i] for i in range(hh)])))
date_id = [day_id, week_id, month_id]
N = y.shape[0]

In [ ]:
##学習データとテストデータに分割
#インデックスの作成
N1 = int(N*4/5); N2 = int(N*1/5)
index_learn = np.sort(np.random.choice(N, N1, replace=False))
index_test = np.delete(np.arange(N), index_learn)

#学習データを作成
z1 = z[index_learn, ]
y1 = y[index_learn]
d_id1 = d_id[index_learn]
location_id1 = location_id[index_learn]
day_id1 = day_id[index_learn]
week_id1 = week_id[index_learn]
month_id1 = month_id[index_learn]
date_id1 = [day_id[index_learn], week_id[index_learn], month_id[index_learn]]
pt1 = pd.Series(d_id1).value_counts()
pt1 = np.array(pt1.iloc[np.argsort(pt1.index)], dtype="int")

#テストデータを作成
z2 = z[index_test, ]
y2 = y[index_test]
d_id2 = d_id[index_test]
location_id2 = location_id[index_test]
day_id2 = day_id[index_test]
week_id2 = week_id[index_test]
month_id2 = month_id[index_test]
date_id2 = [day_id[index_test], week_id[index_test], month_id[index_test]]
pt2 = pd.Series(d_id2).value_counts()
pt2 = np.array(pt2.iloc[np.argsort(pt2.index)], dtype="int")

In [ ]:
#インデックスを作成
d_list = [i for i in range(hh)]
d_vec = [i for i in range(hh)]
location_list = [i for i in range(location)]
location_vec = [i for i in range(location)]
date_list = [i for i in range(m)]
date_vec = [i for i in range(m)]
location_n = np.repeat(0, location)
for i in range(hh):
    d_list[i] = np.array(np.where(d_id1==i)[0], dtype="int")
    d_vec[i] = np.repeat(1.0, d_list[i].shape[0])
for j in range(location):
    location_list[j] = np.array(np.where(location_id1==j)[0], dtype="int")
    location_vec[j] = np.repeat(1.0, location_list[j].shape[0])
    location_n[j] = location_list[j].shape[0]
for i in range(mode):
    temp_list = [j for j in range(date_n[i])]
    temp_vec = [j for j in range(date_n[i])]
    for j in range(date_n[i]):
        temp_list[j] = np.array(np.where(date_id1[i]==j)[0], dtype="int")
        temp_vec[j] = np.repeat(1.0, temp_list[j].shape[0])
    date_list[i] = temp_list
    date_vec[i] = temp_vec

In [ ]:
####可変基底非負値テンソル分解のパラメータを推定####
##アルゴリズムの設定
R = 2000
keep = 2
burnin = int(500/keep)
iter = 0
disp = 10

In [5]:
##事前分布の設定
gamma = np.array([1.5, 1.0, 0.5, 0.5])
alpha1 = 0.1; beta1 = 0.1
alpha2 = 0.1; beta2 = 0.1
alpha3 = 0.1; beta3 = 1.0

In [6]:
##パラメータの真値
#モデルパラメータの真値
omega = omegat.copy()
theta_u = thetat_u.copy()
theta_v = thetat_v.copy()
theta_d = thetat_d.copy()

#トピックの真値
Zi = z1.copy()
z_vec = np.dot(Zi, np.arange(mode+1))

NameError: name 'omegat' is not defined

In [28]:
##パラメータ初期値
#事前分布の初期値
alpha = 0.5
beta = 1.0

#モデルパラメータの初期値
omega = np.random.dirichlet(gamma, hh)
theta_u = np.random.gamma(alpha, 1/beta, hh*k).reshape(hh, k)
theta_v = np.random.gamma(alpha, 1/beta, location*k).reshape(location, k)
theta_d = [j for j in range(mode+1)]
for j in range(mode):
    theta_d[j] = np.random.gamma(alpha, 1/beta, date_n[j]*k).reshape(date_n[j], k)
theta_d[mode] = np.repeat(1.0, k)

#トピックの初期値
res = rmnom(omega[d_id1, ], N1, mode+1, np.arange(N1), 1)
z_vec = np.array(res[0], dtype="int8")
Zi = np.array(res[1].todense(), dtype="int8")

In [134]:
##パラメータの格納用配列
#モデルパラメータの格納用配列
OMEGA = np.zeros((hh, mode+1, int(R/keep)))
THETA_U = np.zeros((hh, k, int(R/keep)))
THETA_V = np.zeros((location, k, int(R/keep)))
THETA_D1 = np.zeros((date_n[0], k, int(R/keep)))
THETA_D2 = np.zeros((date_n[1], k, int(R/keep)))
THETA_D3 = np.zeros((date_n[2], k, int(R/keep)))

#潜在変数の格納用配列
S = np.repeat(0.0, N1)
SEG = np.zeros((N1, mode+1), dtype="int")

In [ ]:
##対数尤度の基準値


In [104]:
####ギブスサンプリングでパラメータをサンプリング####
##潜在変数sをサンプリング
#潜在変数sの割当確率
mu = np.zeros((N1, mode+1))
mu_deploy = [j for j in range(mode+1)]
joint_theta = theta_u[d_id1, ] * theta_v[location_id1, ]
for j in range(mode+1):
    if j < mode:
        mu_deploy[j] = joint_theta * theta_d[j][date_id1[j], ]
        mu[:, j] = np.dot(mu_deploy[j], k_vec)
    else:
        mu_deploy[j] = joint_theta.copy()
        mu[:, j] = np.dot(mu_deploy[j], k_vec)
Lambda = np.sum(Zi * mu, axis=1)
Prob = 1 - np.exp(-Lambda)

#潜在変数sから新しい変数を作成
s = np.random.binomial(1, Prob, N1)
y_new = s * y1


##トピックをサンプリング
#トピックの割当確率
Posterior = omega[d_id1, ] * scipy.stats.poisson.pmf(y_new[:, np.newaxis], mu)
Prob = Posterior / np.sum(Posterior, axis=1)[:, np.newaxis]

#多項分布からトピックをサンプリング
res = rmnom(Prob, N1, mode+1, np.arange(N1), 1)
z_vec = np.array(res[0], dtype="int8")
Zi = np.array(res[1].todense(), dtype="int8")

In [105]:
##ユーザーのテンソル分解のパラメータをサンプリング
#補助変数deltaを更新
theta_location = theta_v[location_id1, ]
theta_date = np.zeros((N1, k)); theta_date[Zi[:, mode]==1, ] = 1.0
lambda_deploy = np.zeros((N1, k))
for j in range(mode+1):
    z_target = Zi[:, j][:, np.newaxis]
    lambda_deploy += mu_deploy[j] * z_target
    if j < mode:
        theta_date += theta_d[j][date_id1[j], ] * z_target
delta = lambda_deploy / np.sum(Zi * mu, axis=1)[:, np.newaxis]

#ガンマ分布の事後分布のパラメータ
delta_y = delta * y_new[:, np.newaxis]
W1 = np.zeros((hh, k)); W2 = np.zeros((hh, k))
for i in range(hh):
    index = d_list[i]
    W1[i, ] = np.dot(delta_y[index, ].T, d_vec[i])
    W2[i, ] = np.dot((theta_location[index, ] * theta_date[index, ]).T, d_vec[i])
    
#ガンマ分布からパラメータをサンプリング
theta_u = np.random.gamma(W1+alpha1, 1/(W2+beta1))
theta_user = theta_u[d_id1, ]

In [109]:
##場所のテンソル分解のパラメータをサンプリング
#補助変数deltaを更新
lambda_deploy = theta_user * theta_location * theta_date
delta = lambda_deploy / np.sum(lambda_deploy, axis=1)[:, np.newaxis]

#ガンマ分布の事後分布のパラメータ
delta_y = delta * y_new[:, np.newaxis]
H1 = np.zeros((location, k)); H2 = np.zeros((location, k))
for i in range(location):
    index = location_list[i]
    H1[i, ] = np.dot(delta_y[index, ].T, location_vec[i])
    H2[i, ] = np.dot((theta_user[index, ] * theta_date[index, ]).T, location_vec[i])
    
#ガンマ分布からパラメータをサンプリング
theta_v = np.random.gamma(H1+alpha2, 1/(H2+beta2))
theta_location = theta_v[location_id1, ]

In [132]:
##時間のテンソル分解のパラメータをサンプリング
#補助変数deltaをサンプリング
lambda_deploy = theta_user * theta_location * theta_date
delta = lambda_deploy / np.sum(lambda_deploy, axis=1)[:, np.newaxis]

#ガンマ分布の事後分布のパラメータ
delta_y = delta * y_new[:, np.newaxis]
for i in range(mode):
    C1 = np.zeros((date_n[i], k)); C2 = np.zeros((date_n[i], k))
    for j in range(date_n[i]):
        index = date_list[i][j]
        C1[j, ] = np.dot(delta_y[index, ].T, Zi[index, i])
        C2[j, ] = np.dot((theta_user[index, ] * theta_location[index, ]).T, Zi[index, i])

    #ガンマ分布からパラメータをサンプリング
    theta_d[i] = np.random.gamma(C1+alpha3, 1/(C2+beta3))